# Introduction to centralities

In this notebook we implemented centralities and we are testing them. 
All the finished implementations can be imported directly from `src/functions.py`.

## Implementations

In [7]:
import numpy as np
import awkward as ak
import uproot

In [74]:
def calcWeight(mode, cluster1, cluster2,n):
    E1 = cluster1.energy()
    E2 = cluster2.energy()
    dx = cluster2.x()-cluster1.x()
    dy = cluster2.y()-cluster1.y()
    dz = cluster2.z()-cluster1.z()
    dist = sqrt(np.power(dx,2)+np.power(dy,2)+np.power(dz,2))
    """
    Default is not weighted
    Method 1: max(E1, E2)
    Method 2: |E1 - E2|
    Method 3: (d)^(-n)
    Method 4: (method 1 or 2)*(method 3)
    """
    weight = 1;
    if(mode==1):
        weight = max(E1, E2)
    elif(mode==2):
        weight = np.abs(E1-E2)
    elif(mode==3):
        weight = np.power(dist,-n)
    elif(mode==4):
        weight = np.abs(E1-E2)*np.power(dist,-n)
        
    return weight

In [12]:
def adjM(trackster,isDirected=False):
    adj = np.zeros(len(trackster.nodes),len(trackster.nodes))
    for edge in trackster.edges:
        #cluster1=layerCluster(edge[0])
        #cluster2=layerCluster(edge[1])
        weight=calcWeight(wtMode,cluster1,cluster2,1)
        adj[edge[0],edge[1]] = weight
        if(not isDirected):
            adj[edge[1],edge[0]] = weight
    maxVal=adj.max()
    return adj/maxVal

In [69]:
def centralityEigen(adj):
    rows,columns= adj.shape
    eigvals,vecl= np.linalg.eig(adj)
    print(eigvals)
    i=np.argmax(np.abs(eigvals)) 
    c_eig= vecl[:,i]
    if(c_eig[0]<0):
        c_eig *=-1
    print(vecl)
    norm=np.linalg.norm(c_eig)
    return c_eig/norm
    

In [70]:
def centralityKatz(adj):
    rows,columns= adj.shape
    Id=np.identity(rows)
    eigvals,vecl= np.linalg.eig(adj)
    print(eigvals)
    i=np.argmax(np.abs(eigvals)) 
    alpha= 0.9/eigvals[i]
    c_katz=(np.linalg.inv(Id-alpha*adj.T)-Id)@np.ones((rows)).T
    print(vecl)
    norm=np.linalg.norm(c_katz)
    return c_katz/norm
    

In [71]:
def centralityPageRank(adj,df):
    rows,columns= adj.shape
    m_ones=np.ones((rows,columns))
    m_pr=df*adj+(1-df)*m_ones/rows
    eigvals,vecl= np.linalg.eig(m_pr)
    print(eigvals)
    i=np.argmax(np.abs(eigvals)) 
    c_pr= vecl[:,i]
    if(c_pr[0]<0):
        c_pr *=-1
    print(vecl)
    norm=np.linalg.norm(c_pr)
    return c_pr/norm
    

## Testing

In [67]:
adj=np.array([[0,1,0,0],[1,0,1,1],[0,1,0,1],[0,1,1,0]])
print(adj)

[[0 1 0 0]
 [1 0 1 1]
 [0 1 0 1]
 [0 1 1 0]]


In [72]:
print('Eigen:')
print(centralityEigen(adj))
print("Katz:")
print(centralityKatz(adj))
print("PageRank:")
print(centralityPageRank(adj,0.85))

Eigen:
[-1.4811943   0.31110782  2.17008649 -1.        ]
[[ 5.05936655e-01  8.15224745e-01  2.81845199e-01 -1.57009246e-16]
 [-7.49390492e-01  2.53622791e-01  6.11628457e-01  1.57009246e-16]
 [ 3.02028137e-01 -3.68160356e-01  5.22720726e-01 -7.07106781e-01]
 [ 3.02028137e-01 -3.68160356e-01  5.22720726e-01  7.07106781e-01]]
[0.2818452  0.61162846 0.52272073 0.52272073]
Katz:
[-1.4811943   0.31110782  2.17008649 -1.        ]
[[ 5.05936655e-01  8.15224745e-01 -2.81845199e-01 -1.57009246e-16]
 [-7.49390492e-01  2.53622791e-01 -6.11628457e-01  1.57009246e-16]
 [ 3.02028137e-01 -3.68160356e-01 -5.22720726e-01 -7.07106781e-01]
 [ 3.02028137e-01 -3.68160356e-01 -5.22720726e-01  7.07106781e-01]]
[0.28015824 0.61821575 0.51928829 0.51928829]
PageRank:
[-1.25436321  0.26825896  1.98610425 -0.85      ]
[[ 5.01340873e-01 -8.12532312e-01  2.97403045e-01 -9.99849633e-17]
 [-7.55015968e-01 -2.42930044e-01  6.09045057e-01  1.03780520e-16]
 [ 2.98837930e-01  3.74684024e-01  5.19911313e-01 -7.07106781e-